In [1]:
# importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score, train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

#Otros 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_raw = pd.read_feather('./data/pivot_data')

df_raw.index = df_raw['index']
df_raw.drop('index', axis=1, inplace=True)

df_raw

,argentino,asistencia_respiratoria,confirmados,cuidado_intensivo,edad,fallecidos,mayor_65,mayor_65_internacion,presentan_sintomas,privado,publico,internacion,sexo_F,sexo_M
index,,,,,,,,,,,,,,
2020-03-01,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
2020-03-02,1,0,1,0,43.000000,0,0,0,1,1,0,1,0,1
2020-03-03,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
2020-03-04,0,0,0,0,0.000000,0,0,0,0,0,0,0,0,0
2020-03-05,5,1,5,1,50.600000,1,2,1,5,3,2,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-26,676,7,692,9,44.251445,10,121,29,359,393,299,53,348,344
2020-09-27,556,2,567,6,40.890212,7,76,21,335,307,260,45,274,293
2020-09-28,764,3,785,6,41.391614,9,113,26,431,480,305,53,399,386


## Pruebo con las principales features primero

In [3]:
cols = ['fallecidos', 'mayor_65', 'mayor_65_internacion', 'edad', 'sexo_M', 'confirmados']
df = df_raw[cols]

In [4]:
#df['fallecidos_mañana'] = df.fallecidos.shift(periods=-1, fill_value=0)
df.head()

,fallecidos,mayor_65,mayor_65_internacion,edad,sexo_M,confirmados
index,,,,,,
2020-03-01,0,0,0,0.0,0,0
2020-03-02,0,0,0,43.0,1,1
2020-03-03,0,0,0,0.0,0,0
2020-03-04,0,0,0,0.0,0,0
2020-03-05,1,2,1,50.6,4,5


## Desplazo fallecidos

In [5]:
df_falle = df_raw[cols].copy()
shift = 1
df_falle['future'] = df.fallecidos.shift(shift, fill_value=0)
df_falle = df_falle[['future', *cols]]
df_falle.tail(8)

,future,fallecidos,mayor_65,mayor_65_internacion,edad,sexo_M,confirmados
index,,,,,,,
2020-09-23,11,16,123,28,41.261972,422,891
2020-09-24,16,11,121,31,40.528175,463,908
2020-09-25,11,15,135,35,42.123344,447,931
2020-09-26,15,10,121,29,44.251445,344,692
2020-09-27,10,7,76,21,40.890212,293,567
2020-09-28,7,9,113,26,41.391614,386,785
2020-09-29,9,7,135,25,42.636799,455,904
2020-09-30,7,11,133,28,42.082355,427,852


In [6]:
df_aux = df.reset_index()
df_aux.drop('index', axis=1, inplace=True)

## Desplazo la salida fallecidos

### Pruebo TimeSeriesSplit

In [7]:
res = pd.DataFrame({'model':[], 'shift':[], 'r2':[], 'mse':[], 'mae':[]})

In [8]:
xtrain = df_falle.loc[df_falle.index < pd.Timestamp('2020-09-01')].drop(['future'], axis = 1)
ytrain = df_falle.loc[df_falle.index < pd.Timestamp('2020-09-01'), 'future']

xtest = df_falle.loc[(df_falle.index >= pd.Timestamp('2020-09-01')) & (df_falle.index < pd.Timestamp('2020-09-15'))].drop(['future'], axis = 1)
ytest = df_falle.loc[(df_falle.index >= pd.Timestamp('2020-09-01')) & (df_falle.index < pd.Timestamp('2020-09-15')), 'future']

## Escalo los datos

In [9]:
scaler = StandardScaler()
xtrain_scal = scaler.fit_transform(xtrain)
xtest_scal = scaler.transform(xtest)

In [10]:
# Pruebas
m = Ridge(alpha=0.1)

tscv = TimeSeriesSplit(n_splits=3)

cv_results = cross_val_score(m, xtrain_scal, ytrain, cv=tscv, scoring='r2')
cv_results

array([ 0.36196477,  0.4753982 , -0.61409094])

In [11]:
params_RR = {
    'alpha': [0.01, 0.1, 1]
}
params_LR = {
    'fit_intercept': [True, False]
}

params_KNR = {
    'n_neighbors': [2, 3, 5, 7],
    'p': [1, 2],
    'leaf_size': [2, 3, 5, 10, 30, 50]
}

params_SVR = {
    'kernel': ['linear', 'poly', ' rbf', 'sigmoid'],
    'gamma': ['scale', 0.01, 0.1, 1],
    'C': [10, 50, 60],
    'degree': [2, 3]
}

In [12]:
r2 = 0
mse = 0
mae = 0

models = {'LR': LinearRegression(), 'KNR': KNeighborsRegressor(n_jobs=5), 'SVR': SVR(), 'RR': Ridge()}
params = {'LR': params_LR, 'KNR': params_KNR, 'SVR': params_SVR, 'RR': params_RR}

for name, model in models.items():
    
    gs = GridSearchCV(estimator = model, cv = tscv, param_grid=params[name], scoring = 'r2')
    gs.fit(xtrain_scal, ytrain)
    print(f'{name}: {gs.best_params_}')
    
    ypred = gs.best_estimator_.predict(xtest_scal)
    
    r2 = r2_score(ytest, ypred)
    mse = mean_squared_error(ytest, ypred)
    mae = mean_absolute_error(ytest, ypred)
    
    res = res.append({'model':name, 'r2':r2, 'shift':shift, 'mse': mse, 'mae': mae}, ignore_index=True)

LR: {'fit_intercept': True}
KNR: {'leaf_size': 2, 'n_neighbors': 3, 'p': 2}
SVR: {'C': 60, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
RR: {'alpha': 0.1}


In [13]:
res.sort_values('r2', ascending=False)

,model,shift,r2,mse,mae
1,KNR,1.0,0.547466,31.079365,4.619048
2,SVR,1.0,0.271766,50.014041,5.917091
3,RR,1.0,0.263805,50.560815,6.035530
0,LR,1.0,0.246215,51.768885,6.179572
